In [ ]:
pip install pyspark
pip install findspark

In [7]:
import pyspark
import findspark
findspark.init()
from pyspark.sql.functions import*
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VideoGames-Sales-Analysis").master("local[*]").getOrCreate()

In [9]:
from google.colab import files
uploaded = files.upload()

Saving vgsales.csv to vgsales.csv


In [10]:
df = spark.read.csv("vgsales.csv" , inferSchema = True , header = True)
df.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

In [11]:
#Dataframe's dimension
print("Lines : " , df.count() , " , Columns : " , len(df.columns))

Lines :  16598  , Columns :  11


In [12]:
#The type of the dataframe
type(df)

pyspark.sql.dataframe.DataFrame

In [13]:
#The first three lines
df.take(3)

[Row(Rank=1, Name='Wii Sports', Platform='Wii', Year='2006', Genre='Sports', Publisher='Nintendo', NA_Sales=41.49, EU_Sales=29.02, JP_Sales=3.77, Other_Sales=8.46, Global_Sales=82.74),
 Row(Rank=2, Name='Super Mario Bros.', Platform='NES', Year='1985', Genre='Platform', Publisher='Nintendo', NA_Sales=29.08, EU_Sales=3.58, JP_Sales=6.81, Other_Sales=0.77, Global_Sales=40.24),
 Row(Rank=3, Name='Mario Kart Wii', Platform='Wii', Year='2008', Genre='Racing', Publisher='Nintendo', NA_Sales=15.85, EU_Sales=12.88, JP_Sales=3.79, Other_Sales=3.31, Global_Sales=35.82)]

In [14]:
#The data's schema
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [15]:
#Checking for columns types
df.dtypes

[('Rank', 'int'),
 ('Name', 'string'),
 ('Platform', 'string'),
 ('Year', 'string'),
 ('Genre', 'string'),
 ('Publisher', 'string'),
 ('NA_Sales', 'double'),
 ('EU_Sales', 'double'),
 ('JP_Sales', 'double'),
 ('Other_Sales', 'double'),
 ('Global_Sales', 'double')]

In [16]:
#Checking for duplicates values
s = df.groupBy(df.columns).count().where("count > 1")
print("Duplicates values : " , s.count())

Duplicates values :  0


In [17]:
#Checking for missing values
df.select([count(when(isnan(c) | col(c).isNull(), c)). alias(c) for c in df.columns]).show()

+----+----+--------+----+-----+---------+--------+--------+--------+-----------+------------+
|Rank|Name|Platform|Year|Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+----+--------+----+-----+---------+--------+--------+--------+-----------+------------+
|   0|   0|       0|   0|    0|        0|       0|       0|       0|          0|           0|
+----+----+--------+----+-----+---------+--------+--------+--------+-----------+------------+



In [18]:
#Statistics of the dataframe
df.describe().show()

+-------+-----------------+--------------------+--------+------------------+--------+---------------+-------------------+------------------+-------------------+--------------------+------------------+
|summary|             Rank|                Name|Platform|              Year|   Genre|      Publisher|           NA_Sales|          EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+-----------------+--------------------+--------+------------------+--------+---------------+-------------------+------------------+-------------------+--------------------+------------------+
|  count|            16598|               16598|   16598|             16598|   16598|          16598|              16598|             16598|              16598|               16598|             16598|
|   mean|8300.605253645017|              1942.0|  2600.0|2006.4064433147546|    NULL|           NULL|0.26466742981084057|0.1466520062658483|0.07778166044101108|0.048063019640913515|  0.53744065550

In [19]:
df.filter(df['Year'] == "N/A").show()

+----+--------------------+--------+----+---------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|    Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+---------+--------------------+--------+--------+--------+-----------+------------+
| 180|     Madden NFL 2004|     PS2| N/A|   Sports|     Electronic Arts|    4.26|    0.26|    0.01|       0.71|        5.23|
| 378|    FIFA Soccer 2004|     PS2| N/A|   Sports|     Electronic Arts|    0.59|    2.36|    0.04|       0.51|        3.49|
| 432|LEGO Batman: The ...|     Wii| N/A|   Action|Warner Bros. Inte...|    1.86|    1.02|     0.0|       0.29|        3.17|
| 471|wwe Smackdown vs....|     PS2| N/A| Fighting|                 N/A|    1.57|    1.02|     0.0|       0.41|         3.0|
| 608|      Space Invaders|    2600| N/A|  Shooter|               Atari|    2.36|    0.14|     0.0|       0.03|        2.53|


In [20]:
#We replace the N/A values by the string "Not Specify" in all the dataframe spark
NewDf = df.replace("N/A" , "Not Specify")

In [21]:
NewDf.filter(df['Year'] == "N/A").show()

+----+--------------------+--------+-----------+---------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|       Year|    Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+-----------+---------+--------------------+--------+--------+--------+-----------+------------+
| 180|     Madden NFL 2004|     PS2|Not Specify|   Sports|     Electronic Arts|    4.26|    0.26|    0.01|       0.71|        5.23|
| 378|    FIFA Soccer 2004|     PS2|Not Specify|   Sports|     Electronic Arts|    0.59|    2.36|    0.04|       0.51|        3.49|
| 432|LEGO Batman: The ...|     Wii|Not Specify|   Action|Warner Bros. Inte...|    1.86|    1.02|     0.0|       0.29|        3.17|
| 471|wwe Smackdown vs....|     PS2|Not Specify| Fighting|         Not Specify|    1.57|    1.02|     0.0|       0.41|         3.0|
| 608|      Space Invaders|    2600|Not Specify|  Shooter|               Ata

### **The preprocess of data was done , now let's begin our analysis.**

In [22]:
#1-Total sales in each country and in the world
from pyspark.sql import functions as F
df_total_sales = NewDf.withColumn('Country', F.lit('NA_Sales')) \
             .withColumn('Total Sales', F.col('NA_Sales')) \
             .union(NewDf.withColumn('Country', F.lit('EU_Sales')) \
                        .withColumn('Total Sales', F.col('EU_Sales'))) \
             .union(NewDf.withColumn('Country', F.lit('JP_Sales')) \
                        .withColumn('Total Sales', F.col('JP_Sales'))) \
             .union(NewDf.withColumn('Country', F.lit('Other_Sales')) \
                        .withColumn('Total Sales', F.col('Other_Sales'))) \
              .union(NewDf.withColumn('Country', F.lit('Global_Sales')) \
                        .withColumn('Total Sales', F.col('Global_Sales'))) \
             .groupBy('Country').agg(F.sum('Total Sales').alias('Total Sales'))
df_total_sales.show()

+------------+------------------+
|     Country|       Total Sales|
+------------+------------------+
|    NA_Sales| 4392.950000000332|
|    EU_Sales|  2434.13000000055|
|    JP_Sales|1291.0199999999018|
| Other_Sales| 797.7499999998826|
|Global_Sales| 8920.440000001283|
+------------+------------------+



 AMERICA is the first country with the most video games sales , that's very amazing 'cause JAPAN which is famous about manga tv comes at the third position.

In [23]:
#2-The 10 first years with the most sale
df_sales_years = NewDf.groupby("Year").agg(F.sum("Global_Sales").alias("Total_Global_Sales")).sort(F.desc("Total_Global_Sales"))
df_sales_years_10 = df_sales_years.limit(10)
df_sales_years_10.show()

+----+------------------+
|Year|Total_Global_Sales|
+----+------------------+
|2008| 678.8999999999952|
|2009| 667.2999999999947|
|2007| 611.1299999999934|
|2010| 600.4499999999948|
|2006| 521.0399999999917|
|2011| 515.9899999999969|
|2005| 459.9399999999976|
|2004|419.30999999999864|
|2002| 395.5199999999983|
|2013|368.10999999999865|
+----+------------------+



OMGGG , the good times!!! 2008 is the year with the most global sales. Note also that the global_sales over years was very in constantly changing.

In [24]:
#3-The top 5 popular video  game sale
df_popular_vgames = NewDf.groupby("Name").agg(F.sum("Global_Sales").alias("Total_Global_Sales")).sort(F.desc("Total_Global_Sales"))
df_popular_vgames_5 = df_popular_vgames.limit(5)
df_popular_vgames_5.show()

+------------------+------------------+
|              Name|Total_Global_Sales|
+------------------+------------------+
|        Wii Sports|             82.74|
|Grand Theft Auto V|             55.92|
| Super Mario Bros.|             45.31|
|            Tetris|             35.84|
|    Mario Kart Wii|             35.82|
+------------------+------------------+



Wii sport , is a video game sport , the most like in the world.

In [25]:
#4-The most represented gender
genre_count = NewDf.groupBy("Genre").count().orderBy("count", ascending=False)
genre_count.show()

+------------+-----+
|       Genre|count|
+------------+-----+
|      Action| 3316|
|      Sports| 2346|
|        Misc| 1739|
|Role-Playing| 1488|
|     Shooter| 1310|
|   Adventure| 1286|
|      Racing| 1249|
|    Platform|  886|
|  Simulation|  867|
|    Fighting|  848|
|    Strategy|  681|
|      Puzzle|  582|
+------------+-----+



Action is the most represented gender , following by sport , in order words , action following by sport and etc.. have the most vgames(already checked).

In [26]:
#5-The most sales genders in the world
df_genre = NewDf.groupby("Genre").agg(F.sum("Global_Sales").alias("Total_Global_Sales")).sort(F.desc("Total_Global_Sales"))
df_genre.show()

+------------+------------------+
|       Genre|Total_Global_Sales|
+------------+------------------+
|      Action|1751.1799999999691|
|      Sports| 1330.929999999988|
|     Shooter|1037.3699999999901|
|Role-Playing| 927.3699999999941|
|    Platform| 831.3699999999974|
|        Misc| 809.9599999999936|
|      Racing| 732.0399999999955|
|    Fighting|448.90999999999923|
|  Simulation|392.19999999999783|
|      Puzzle| 244.9500000000005|
|   Adventure|239.04000000000138|
|    Strategy| 175.1200000000004|
+------------+------------------+



Action remain not only the first gender with the most number of representations but also the first gender wich is the most sale in the world.

In [30]:
#6-Numbers of vgames by publisher
publisher_platform_counts = df.groupBy("Publisher") \
                             .agg(countDistinct("Name").alias("Nb_Vgames")) \
                             .orderBy("Nb_Vgames", ascending=False)
publisher_platform_counts.show()

+--------------------+---------+
|           Publisher|Nb_Vgames|
+--------------------+---------+
|  Namco Bandai Games|      776|
|            Nintendo|      667|
|Konami Digital En...|      640|
|Sony Computer Ent...|      622|
|     Electronic Arts|      606|
|             Ubisoft|      572|
|                Sega|      480|
|          Activision|      417|
|                 THQ|      409|
|               Atari|      280|
|              Capcom|      277|
|          Tecmo Koei|      253|
|Take-Two Interactive|      201|
|             Unknown|      187|
|Microsoft Game St...|      179|
|         Square Enix|      161|
|           505 Games|      149|
|         D3Publisher|      132|
|        Idea Factory|      127|
|   Eidos Interactive|      124|
+--------------------+---------+
only showing top 20 rows



 Namco Bandai Games , follows by Nitendo have the most Vgames , they are the publisher the most widespread in the morld.

In [ ]:
#7-Numbers of Vgames by platform
publisher_platform_counts = df.groupBy("Platform") \
                             .agg(countDistinct("Name").alias("Nb_Vgames")) \
                             .orderBy("Nb_Vgames", ascending=False)
publisher_platform_counts.show()

+--------+---------+
|Platform|Nb_Vgames|
+--------+---------+
|      DS|     2163|
|     PS2|     2161|
|     PS3|     1327|
|     Wii|     1324|
|    X360|     1264|
|     PSP|     1213|
|      PS|     1196|
|      PC|      959|
|      XB|      824|
|     GBA|      822|
|      GC|      556|
|     3DS|      509|
|     PSV|      413|
|     PS4|      336|
|     N64|      319|
|    SNES|      239|
|    XOne|      213|
|     SAT|      173|
|    WiiU|      143|
|    2600|      133|
+--------+---------+
only showing top 20 rows



The platform DS has the most vgames , so we can say that , he has also the most publisher(already checked) in the world, thus DS is the most like platform by users.

In [33]:
#8-The publisher with the most global_sales.
df_publisher = NewDf.groupby("Publisher").agg(F.sum("Global_Sales").alias("Total_Global_Sales")).sort(F.desc("Total_Global_Sales"))
df_publisher.show()

+--------------------+------------------+
|           Publisher|Total_Global_Sales|
+--------------------+------------------+
|            Nintendo|1786.5599999999981|
|     Electronic Arts|1110.3199999999915|
|          Activision| 727.4599999999983|
|Sony Computer Ent...| 607.4999999999989|
|             Ubisoft|474.71999999999935|
|Take-Two Interactive| 399.5399999999996|
|                 THQ| 340.7699999999994|
|Konami Digital En...|  283.639999999998|
|                Sega|272.98999999999927|
|  Namco Bandai Games| 254.0900000000008|
|Microsoft Game St...|245.79000000000005|
|              Capcom|200.89000000000001|
|               Atari|157.22000000000025|
|Warner Bros. Inte...|153.89000000000013|
|         Square Enix|145.18000000000026|
|Disney Interactiv...|119.96000000000004|
|   Eidos Interactive| 98.97999999999998|
|           LucasArts| 87.34000000000003|
|  Bethesda Softworks| 82.14000000000003|
|        Midway Games| 69.84999999999994|
+--------------------+------------

OMGGG , Nitendo , totally love it , the publisher with the highest global_sale , he is the publisher the most attracted by the users in the world.